# `hops-util-py` Integration Tests

This notebook can be converted to a python file and submitted as a spark job for integration tests

## Imports

In [1]:
from hops import experiment, hdfs, tensorboard, devices, kafka, featurestore, tls, util, serving
import stat
import os
import shutil
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, IntegerType, FloatType
import pandas as pd
import numpy as np
import datetime
import time
from pyspark.sql import DataFrame
from petastorm.unischema import dict_to_spark_row, Unischema, UnischemaField
from petastorm.codecs import ScalarCodec, CompressedImageCodec, NdarrayCodec
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType
from pyspark.sql import SparkSession
import tensorflow as tf
import sys
import random
from confluent_kafka import Producer, Consumer, KafkaError

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
48,application_1558113321244_0050,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


## Experiments Tests

##### Test Custom Experiments

- `experiment.begin()`
- `experiment.end()`

In [ ]:
experiment.begin(name='some custom thing 1', local_logdir=False)
assert tensorboard.logdir() != None
assert "hdfs://" in tensorboard.logdir()
pi = 1+3+0.14
experiment.end(pi)

In [ ]:
experiment.begin(name='some custom thing 2', local_logdir=True, description='i am making custom exp on hops')
assert tensorboard.logdir() != None
assert "hdfs://" not in tensorboard.logdir()
pi = 1337
experiment.end(pi)

##### Test `experiment.launch`

In [ ]:
def wrapper():
    assert tensorboard.logdir() != None
    assert devices.get_num_gpus() >= 0
    assert hdfs.project_path() == hdfs.project_path(hdfs.project_name())
    if tensorboard.local_logdir_bool:
        assert "hdfs://" not in tensorboard.logdir()
    else:
        assert "hdfs://" in tensorboard.logdir()

In [ ]:
experiment.launch(wrapper, local_logdir=True)

In [ ]:
experiment.launch(wrapper, description='very interesting description', local_logdir=False)

In [ ]:
def parameter_wrapper(a, b):
    assert tensorboard.logdir() != None
    assert devices.get_num_gpus() >= 0
    if tensorboard.local_logdir_bool:
        assert "hdfs://" not in tensorboard.logdir()
    else:
        assert "hdfs://" in tensorboard.logdir()
    return a + b

In [ ]:
experiment.launch(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, local_logdir=True)

In [ ]:
experiment.launch(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, description='very interesting description', local_logdir=False)

##### Test Parallel Experiments `experiment.grid_search`, `experiment.random_search`, `experiment.differential_evolution`

In [ ]:
experiment.grid_search(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, direction='min', name='test')

In [ ]:
experiment.grid_search(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, direction='max', name='test', local_logdir=True)

In [ ]:
experiment.random_search(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, direction='max', name='test')

In [ ]:
experiment.random_search(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, direction='min', samples=3, name='test', local_logdir=True)

In [ ]:
logdir1, result_dict1 = experiment.differential_evolution(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, local_logdir=True, direction='max', generations=8, population=12)
assert int(result_dict1['a']) == 2 and int(result_dict1['b']) == 1

In [ ]:
logdir2, result_dict2 = experiment.differential_evolution(parameter_wrapper, {'a': [1,2], 'b': [-1,1]}, generations=8, population=10, direction='min')
assert int(result_dict2['a']) == 1 and int(result_dict2['b']) == -1

## HopsFS Tests

##### Test HopsFS operations

- `hdfs.project_user()`
- `hdfs.project_name()`
- `hdfs.project_path()`
- `hdfs.exists()`
- `hdfs.load()`
- `hdfs.copy_to_hdfs()`
- `hdfs.copy_to_local()`
- `hdfs.ls()`
- `hdfs.lsl()`
- `hdfs.glob()`
- `hdfs.cp()`
- `hdfs.rmr()`
- `hdfs.rename()`
- `hdfs.stat()`
- `hdfs.isdir()`
- `hdfs.isfile()`
- `hdfs.add_module()`
- `hdfs.delete()`
- `hdfs.get_plain_path()`

In [ ]:
project_user = hdfs.project_user()
project_name = hdfs.project_name()
assert project_name in project_user
project_path = hdfs.project_path()
assert project_name in project_path

In [ ]:
logs_README = hdfs.load("Logs/README.md")
assert len(logs_README) > 0

In [ ]:
hdfs.dump("test", "Logs/README_dump_test.md")
assert hdfs.exists("Logs/README_dump_test.md")

In [ ]:
logs_README_dumped = hdfs.load("Logs/README_dump_test.md")
assert logs_README_dumped.decode("utf-8") == "test"

In [ ]:
# copy_to_hdfs file relative path

with open('upload.txt', 'w') as f:
    f.write("first upload")
hdfs.copy_to_hdfs("upload.txt", "Resources")
assert hdfs.exists("Resources/upload.txt")
hdfs_copied_file = hdfs.load("Resources/upload.txt")
assert "first upload" == hdfs_copied_file.decode("utf-8"), "first content does not match"

with open('upload.txt', 'w') as f:
    f.write("second upload")
hdfs.copy_to_hdfs("upload.txt", "Resources", overwrite=True)
assert hdfs.exists("Resources/upload.txt")
hdfs_copied_file = hdfs.load("Resources/upload.txt")
assert "second upload" == hdfs_copied_file.decode("utf-8"), "second content does not match"

try:
    hdfs.copy_to_hdfs("upload.txt", "Resources")
    assert False
except IOError:
    pass

hdfs.rmr("Resources/upload.txt")
os.remove("upload.txt")

In [ ]:
# copy_to_hdfs file absolute path

with open('upload_absolute.txt', 'w') as f:
    f.write("first upload")
hdfs.copy_to_hdfs(os.getcwd() + "/upload_absolute.txt", "Resources")
assert hdfs.exists("Resources/upload_absolute.txt")
hdfs_copied_file = hdfs.load("Resources/upload_absolute.txt")
assert "first upload" == hdfs_copied_file.decode("utf-8"), "first content does not match"

with open('upload_absolute.txt', 'w') as f:
    f.write("second upload")
hdfs.copy_to_hdfs(os.getcwd() + "/upload_absolute.txt", "Resources", overwrite=True)
assert hdfs.exists("Resources/upload_absolute.txt")
hdfs_copied_file = hdfs.load("Resources/upload_absolute.txt")
assert "second upload" == hdfs_copied_file.decode("utf-8"), "second content does not match"

try:
    hdfs.copy_to_hdfs("upload_absolute.txt", "Resources")
    assert False
except IOError:
    pass

hdfs.rmr("Resources/upload_absolute.txt")
os.remove("upload_absolute.txt")

In [ ]:
# copy_to_hdfs directory relative path

if not os.path.exists("upload_dir"):
    os.mkdir("upload_dir")

assert not hdfs.exists("Resources/upload_dir")
with open('upload_dir/upload.txt', 'w') as f:
    f.write("first upload")
hdfs.copy_to_hdfs("upload_dir", "Resources")
hdfs_copied_file = hdfs.load("Resources/upload_dir/upload.txt")
assert hdfs.exists("Resources/upload_dir")
with open('upload_dir/upload.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "first content compare failed"

with open('upload_dir/upload.txt', 'w') as f:
    f.write("second upload")
hdfs.copy_to_hdfs("upload_dir", "Resources", overwrite=True)
hdfs_copied_file = hdfs.load("Resources/upload_dir/upload.txt")
assert hdfs.exists("Resources/upload_dir")
with open('upload_dir/upload.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "second content compare failed"

shutil.rmtree("upload_dir")
hdfs.rmr("Resources/upload_dir")

In [ ]:
# copy_to_hdfs directory absolute path

if not os.path.exists("upload_dir_absolute"):
    os.mkdir("upload_dir_absolute")
    
assert not hdfs.exists("Resources/upload_dir_absolute")
with open('upload_dir_absolute/upload.txt', 'w') as f:
    f.write("first upload")
hdfs.copy_to_hdfs(os.getcwd() + "/upload_dir_absolute", "Resources")
hdfs_copied_file = hdfs.load("Resources/upload_dir_absolute/upload.txt")
assert hdfs.exists("Resources/upload_dir_absolute")
with open('upload_dir_absolute/upload.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "first content compare failed"

with open('upload_dir_absolute/upload.txt', 'w') as f:
    f.write("second upload")
hdfs.copy_to_hdfs(os.getcwd() + "/upload_dir_absolute", "Resources", overwrite=True)
hdfs_copied_file = hdfs.load("Resources/upload_dir_absolute/upload.txt")
assert hdfs.exists("Resources/upload_dir_absolute")
with open('upload_dir_absolute/upload.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "second content compare failed"

shutil.rmtree("upload_dir_absolute")
hdfs.rmr("Resources/upload_dir_absolute")

In [ ]:
#copy_to_local file

# Download first time
hdfs.dump("initial content", "Resources/somefile.txt")
hdfs.copy_to_local("Resources/somefile.txt")
hdfs_copied_file = hdfs.load("Resources/somefile.txt")
with open('somefile.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "first content compare failed"
first_modified = os.path.getmtime("somefile.txt")

# Download second time
hdfs.copy_to_local("Resources/somefile.txt")
hdfs_copied_file = hdfs.load("Resources/somefile.txt")
with open('somefile.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "second content compare failed"
second_modified = os.path.getmtime("somefile.txt")
assert first_modified == second_modified, "modified time not matching"

# Content changing on disk
hdfs.dump("content changed at some point", "Resources/somefile.txt")
hdfs_new_content = hdfs.load("Resources/somefile.txt")
hdfs.copy_to_local("Resources/somefile.txt")
with open('somefile.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_new_content.decode("utf-8") == local_copied_file, "third content compare failed"
third_modified = os.path.getmtime("somefile.txt")
assert not second_modified == third_modified, "modified time not matching"

# Download last time with overwrite, file should have changed on disk
hdfs.copy_to_local("Resources/somefile.txt", overwrite=True)
hdfs_copied_file = hdfs.load("Resources/somefile.txt")
with open('somefile.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "fourth content compare failed"
fourth_modified = os.path.getmtime("somefile.txt")
assert not third_modified == fourth_modified, "modified time not matching"

# Download again to make sure overwrite did not cause problems
hdfs.copy_to_local("Resources/somefile.txt")
hdfs_copied_file = hdfs.load("Resources/somefile.txt")
with open('somefile.txt', 'r') as f:
    local_copied_file = f.read()
assert hdfs_copied_file.decode("utf-8") == local_copied_file, "fifth content compare failed"
fifth_modified = os.path.getmtime("somefile.txt")
assert fourth_modified == fifth_modified, "modified time not matching"

hdfs.rmr("Resources/somefile.txt")
os.remove("somefile.txt")

In [ ]:
#copy_to_local directory

assert not os.path.exists("Resources")
hdfs.copy_to_local("Resources")
first_modified = os.path.getmtime("Resources")
assert os.path.exists("Resources")
assert os.path.isdir("Resources")

hdfs.copy_to_local("Resources")
second_modified = os.path.getmtime("Resources")
assert first_modified == second_modified

localized_dir = hdfs.copy_to_local("Resources", overwrite=True)
third_modified = os.path.getmtime("Resources")
assert not second_modified == third_modified
num_files_first = len(os.listdir(localized_dir))

# Add a new file, it should also be localized
hdfs.dump("a wild file appeared", "Resources/newfile.txt")
hdfs.copy_to_local("Resources")
fourth_modified = os.path.getmtime("Resources")
assert first_modified == second_modified
num_files_second = len(os.listdir(localized_dir))
assert (num_files_first + 1) == num_files_second
assert not third_modified == fourth_modified

hdfs.rmr("Resources/newfile.txt")
shutil.rmtree("Resources")

In [ ]:
logs_files_md = hdfs.glob("Logs/*.md")
logs_path_names = hdfs.lsl("Logs/")
if hdfs.exists("Logs/test.txt"):
    hdfs.rmr("Logs/test.txt")
assert not hdfs.exists("Logs/test.txt")

In [ ]:
hdfs.dump("dummy", "Resources/test.txt")
hdfs.cp("Resources/test.txt", "Logs/")
logs_files = hdfs.ls("Logs/")
assert "test.txt" in ",".join(logs_files)

In [ ]:
hdfs.mkdir("Logs/test_dir")
assert hdfs.exists("Logs/test_dir")

In [ ]:
logs_files_prior_delete = hdfs.ls("Logs/")
hdfs.rmr("Logs/test_dir")
logs_files_after_delete = hdfs.ls("Logs/")
assert len(logs_files_prior_delete) > len(logs_files_after_delete)

In [ ]:
logs_files_prior_move = hdfs.ls("Logs/")
assert "README_dump_test.md" in ",".join(logs_files_prior_move)

In [ ]:
hdfs.move("Logs/README_dump_test.md", "Logs/README_dump_test2.md")
logs_files_after_move = hdfs.ls("Logs/")
assert "README_dump_test.md" not in ",".join(logs_files_after_move)
assert "README_dump_test2.md" in ",".join(logs_files_after_move)

In [ ]:
logs_files_prior_rename = hdfs.ls("Logs/")
assert "README_dump_test2.md" in ",".join(logs_files_prior_rename)

In [ ]:
hdfs.rename("Logs/README_dump_test2.md", "Logs/README_dump_test.md")
logs_files_after_rename = hdfs.ls("Logs/")
assert "Logs/README_dump_test2.md" not in ",".join(logs_files_after_rename)
assert "Logs/README_dump_test.md" in ",".join(logs_files_after_rename)

In [ ]:
file_stat = hdfs.stat("Logs/README.md")
hdfs.chmod("Logs/README.md", 775)
file_stat = hdfs.stat("Logs/README.md")
assert 775 == file_stat.st_mode

In [ ]:
hdfs.chmod("Logs/README.md", 777)
file_stat = hdfs.stat("Logs/README.md")
assert 777 == file_stat.st_mode

In [ ]:
file_owner = file_stat.st_uid
assert hdfs.exists("Logs/")
assert not hdfs.exists("Not_Existing/neither_am_i")

In [ ]:
assert hdfs.isdir("Resources")
assert not hdfs.isdir("Resources/README.md")

In [ ]:
assert hdfs.isfile("Resources/README.md")
assert not hdfs.isfile("Resources")

In [ ]:
hdfs.dump("def simple():\n\treturn 5", "Resources/my_module.py")
py_path = hdfs.add_module("Resources/my_module.py")
assert py_path in sys.path
import my_module
assert my_module.simple() == 5

In [61]:
plain = hdfs.get_plain_path("hdfs://10.0.2.15:8020/Projects/demo_deep_learning_admin000/Models/")
assert plain == "/Projects/demo_deep_learning_admin000/Models/"

In [66]:
hdfs.mkdir("Logs/test_delete_dir")
assert hdfs.exists("Logs/test_delete_dir")
hdfs.delete("Logs/test_delete_dir")
assert not hdfs.exists("Logs/test_delete_dir")

### Experiment API Tests

##### Test Distributed Training MirroredStrategy (This may fail if not configured MirroredStrategy)

- `experiment.mirrored()`

In [ ]:
def mirrored():
    assert 'TF_CONFIG' in os.environ
    assert devices.get_num_gpus() >= 0

In [ ]:
experiment.mirrored(mirrored, local_logdir=True)

In [ ]:
experiment.mirrored(mirrored, name='mirrortime', description='such desc', local_logdir=False)

##### Test Distributed Training CollectiveAllReduceStrategy (This may fail if not configured CollectiveAllReduceStrategy)

- `experiment.collective_all_reduce()`

In [ ]:
def collective():
    assert 'TF_CONFIG' in os.environ
    assert devices.get_num_gpus() >= 0

In [ ]:
experiment.collective_all_reduce(collective, local_logdir=True)

In [ ]:
experiment.collective_all_reduce(collective, name='collectivetime', description='such desc', local_logdir=False)

##### Test Distributed Training ParameterServerStrategy (This may fail if not configured ParameterServerStrategy)

- `experiment.parameter_server()`

In [ ]:
def ps():
    assert 'TF_CONFIG' in os.environ
    assert devices.get_num_gpus() >= 0

In [ ]:
# TODO: these ps tests does not complete, it waits indefinitely
#experiment.parameter_server(ps, local_logdir=False)

In [ ]:
#experiment.parameter_server(ps, local_logdir=True)

In [ ]:
#experiment.parameter_server(ps, name='mirrortime', description='such desc')

## Feature Store Tests

These tests require that you have the following files in the Resources directory:

- `attendances_features.csv`
- `games_features.csv`
- `players_features.csv`
- `season_scores_features.csv`
- `teams_features.csv`

These files can be downloaded from here: `http://snurran.sics.se/hops/hops-util-py_test/`

##### Test Featurestore Create Feature Group Operations (`featurestore.create_featuregroup()`)

In [ ]:
def load_fs_sample_data():
    resources_path = hdfs.project_path() + "Resources/"
    games_features_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(resources_path + "games_features.csv")
    players_features_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(resources_path + "players_features.csv")
    teams_features_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(resources_path + "teams_features.csv")
    season_scores_features_df = spark.read.format("csv").option("header", "true").option("inferSchema","true").load(resources_path + "season_scores_features.csv")
    attendances_features_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(resources_path + "attendances_features.csv")
    return games_features_df,players_features_df,teams_features_df,season_scores_features_df, attendances_features_df
games_features_df,players_features_df,teams_features_df,season_scores_features_df, attendances_features_df = load_fs_sample_data()

In [ ]:
featurestore.create_featuregroup(
    games_features_df,
    "games_features",
    description="Features of average season scores for football teams"
)

In [ ]:
featurestore.create_featuregroup(
    teams_features_df,
    "teams_features",
    description="a spanish version of teams_features"
)

In [ ]:
featurestore.create_featuregroup(
    season_scores_features_df,
    "season_scores_features",
    description="Features of average season scores for football teams"
)

In [ ]:
featurestore.create_featuregroup(
    attendances_features_df,
    "attendances_features",
    description="Features of average attendance of games of football teams"
)

In [ ]:
teams_features_1_df = featurestore.get_featuregroup("teams_features")
teams_features_2_df = teams_features_1_df.withColumnRenamed(
    "team_id", "equipo_id").withColumnRenamed(
    "team_budget", "equipo_presupuesto").withColumnRenamed(
    "team_position", "equipo_posicion")

In [ ]:
featurestore.create_featuregroup(
    teams_features_2_df,
    "teams_features_spanish",
    description="a spanish version of teams_features",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)

In [ ]:
featurestore.create_featuregroup(
    teams_features_2_df,
    "teams_features_spanish",
    description="a spanish version of teams_features",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    featurestore=featurestore.project_featurestore(),
    featuregroup_version=1,
    job_name=None,
    dependencies=[]
)

In [ ]:
featurestore.create_featuregroup(
    teams_features_2_df,
    "teams_features_spanish",
    description="a spanish version of teams_features",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    featuregroup_version=2
)

In [ ]:
assert "games_features_1" in featurestore.get_featuregroups()
assert "teams_features_1" in featurestore.get_featuregroups()
assert "season_scores_features_1" in featurestore.get_featuregroups()
assert "attendances_features_1" in featurestore.get_featuregroups()
assert "teams_features_spanish_1" in featurestore.get_featuregroups()
assert "teams_features_spanish_2" in featurestore.get_featuregroups()

##### Test Featurestore Utility Operations, 

- `featurestore.get_metadata()`,
- `featurestore.project_featurestore()`, 
- `featurestore.get_latest_featuregroup_version()`, 
- `featurestore.get_features_list()`

In [ ]:
featurestore.get_featurestore_metadata(update_cache=True)

In [ ]:
assert featurestore.project_featurestore() == hdfs.project_name() + "_featurestore"

In [ ]:
assert featurestore.project_featurestore() in featurestore.get_project_featurestores()

In [ ]:
assert len(featurestore.get_project_featurestores()) == 1

In [ ]:
assert featurestore.get_latest_featuregroup_version("teams_features_spanish") == 2

In [ ]:
assert featurestore.get_latest_featuregroup_version("teams_features") == 1

In [ ]:
assert "away_team_id" in featurestore.get_features_list()

In [ ]:
assert "home_team_id" in featurestore.get_features_list()

##### Test Read operations of Features and Feature Groups, 

- `featurestore.get_feature()`, 
- `featurestore.get_features()`, 
- `featurestore.get_featuregroup()`

In [ ]:
tmp = featurestore.get_feature("team_budget")
assert tmp.count() == 50
assert len(tmp.columns) == 1
assert "team_budget" in tmp.columns

In [ ]:
tmp = featurestore.get_feature(
    "team_budget", 
    featurestore=featurestore.project_featurestore(), 
    featuregroup="teams_features", 
    featuregroup_version = 1,
    dataframe_type = "spark"
)
assert tmp.count() == 50
assert len(tmp.columns) == 1
assert "team_budget" in tmp.columns

In [ ]:
tmp = featurestore.get_featuregroup("teams_features")
assert tmp.count() == 50
assert len(tmp.columns) == 3
assert "team_budget" in tmp.columns
assert "team_id" in tmp.columns
assert "team_position" in tmp.columns

In [ ]:
tmp = featurestore.get_featuregroup(
    "teams_features", 
    featurestore=featurestore.project_featurestore(), 
    featuregroup_version = 1,
    dataframe_type = "spark"
)
assert tmp.count() == 50
assert len(tmp.columns) == 3
assert "team_budget" in tmp.columns
assert "team_id" in tmp.columns
assert "team_position" in tmp.columns

In [ ]:
features = ["team_budget", "average_attendance"]
tmp = featurestore.get_features(
    features
)
assert set(features) == set(tmp.columns)
assert tmp.count() == 50
assert len(tmp.columns) == len(features)

In [ ]:
features = ["teams_features_1.team_budget", "attendances_features_1.average_attendance"]
tmp = featurestore.get_features(features)
assert set(["team_budget", "average_attendance"]) == set(tmp.columns)
assert tmp.count() == 50
assert len(tmp.columns) == len(features)

In [ ]:
features = ["team_budget", "average_attendance"]
tmp = featurestore.get_features(
    features,
    featurestore=featurestore.project_featurestore(),
    featuregroups_version_dict={
        "teams_features": 1, 
        "attendances_features": 1
    }
)
assert set(features) == set(tmp.columns)
assert tmp.count() == 50
assert len(tmp.columns) == len(features)

In [ ]:
tmp = featurestore.get_features(
    features,
    featurestore=featurestore.project_featurestore(),
    featuregroups_version_dict={
        "teams_features": 1, 
        "attendances_features": 1
    },
    join_key = "team_id",
    dataframe_type = "spark"
)
assert set(features) == set(tmp.columns)
assert tmp.count() == 50
assert len(tmp.columns) == len(features)

In [ ]:
features = ["team_budget", "average_attendance",
    "team_position", "sum_attendance"
    ]
tmp = featurestore.get_features(
   features
)
assert set(features) == set(tmp.columns)
assert tmp.count() == 50
assert len(tmp.columns) == len(features)

In [ ]:
features = ["team_budget", "team_id"]
tmp = featurestore.get_features(
    features,
    featuregroups_version_dict = {
        "teams_features" : 1
    }
)
assert set(features) == set(tmp.columns)
assert tmp.count() == 50
assert len(tmp.columns) == len(features)

In [ ]:
tmp = featurestore.sql(
    "SELECT team_budget, score " \
    "FROM teams_features_1 JOIN games_features_1 ON " \
    "games_features_1.home_team_id = teams_features_1.team_id")
features = ['team_budget', 'score']
assert set(features) == set(tmp.columns)
assert tmp.count() == 49
assert len(tmp.columns) == len(features)

In [ ]:
tmp = featurestore.sql("SELECT * FROM teams_features_1 WHERE team_position < 5")
assert len(tmp.columns) == 3
assert "team_budget" in tmp.columns
assert "team_id" in tmp.columns
assert "team_position" in tmp.columns
for x in tmp.toPandas()["team_position"].values:
    assert x < 5

In [ ]:
tmp = featurestore.sql("SELECT * FROM teams_features_1 WHERE team_position < 5",
                featurestore=featurestore.project_featurestore(), 
                 dataframe_type = "spark")
assert len(tmp.columns) == 3
assert "team_budget" in tmp.columns
assert "team_id" in tmp.columns
assert "team_position" in tmp.columns
for x in tmp.toPandas()["team_position"].values:
    assert x < 5

#####  Test Insert Operations in Existing Feature Groups, `featurestore.insert_into_featuregroup()`

In [ ]:
sqlContext = SQLContext(spark.sparkContext)
schema = StructType([StructField("equipo_id", IntegerType(), True),
                     StructField("equipo_presupuesto", FloatType(), True),
                     StructField("equipo_posicion", IntegerType(), True)
                        ])
sample_df = sqlContext.createDataFrame([(999, 41251.52, 1), (998, 1319.4, 8), (997, 21219.1, 2)], schema)
insert_count = sample_df.count()
assert insert_count == 3

In [ ]:
spanish_team_features_df = featurestore.get_featuregroup(
    "teams_features_spanish")
pre_insert_count = spanish_team_features_df.count()
assert pre_insert_count == 50

In [ ]:
featurestore.insert_into_featuregroup(
    sample_df, 
    "teams_features_spanish", 
    descriptive_statistics=False, 
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)
spanish_team_features_df_updated = featurestore.get_featuregroup(
    "teams_features_spanish")

after_insert_count = spanish_team_features_df_updated.count()
assert after_insert_count == 53

In [ ]:
featurestore.insert_into_featuregroup(
    sample_df, 
    "teams_features_spanish", 
    featurestore=featurestore.project_featurestore(), 
    featuregroup_version=1, 
    mode="append",
    descriptive_statistics=False, 
    feature_correlation=False, 
    feature_histograms=False,
    cluster_analysis=False, 
    stat_columns=None, 
    num_bins=20, 
    corr_method='pearson',
    num_clusters=5
)

after_insert_count2 = featurestore.get_featuregroup("teams_features_spanish").count()
assert after_insert_count2 == 56

In [ ]:
featurestore.insert_into_featuregroup(
    sample_df, 
    "teams_features_spanish",
    descriptive_statistics=False, 
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    mode="overwrite")

count_after_overwrite = featurestore.get_featuregroup("teams_features_spanish").count()
assert count_after_overwrite == 3

##### Test integration of feature store with Numpy, Pandas and plain Python

In [ ]:
pandas_df = featurestore.get_features(["team_budget", "average_attendance"], dataframe_type="pandas")
assert "team_budget" in pandas_df.columns.values
assert "average_attendance" in pandas_df.columns.values
assert len(pandas_df) == 50
assert len(pandas_df.columns.values) == 2
assert isinstance(pandas_df, pd.DataFrame)

In [ ]:
numpy_df = featurestore.get_features(["team_budget", "average_attendance"], 
                                      dataframe_type="numpy")
assert numpy_df.shape[0] == 50
assert numpy_df.shape[1] == 2
assert isinstance(numpy_df, np.ndarray)

In [ ]:
python_df = featurestore.get_features(["team_budget", "average_attendance"], 
                                      dataframe_type="python")
assert len(python_df) == 50
assert isinstance(python_df, list)

In [ ]:
spark_df = featurestore.get_features(["team_budget", "average_attendance"], 
                                      dataframe_type="spark")
assert spark_df.count() == 50
assert isinstance(spark_df, DataFrame)

In [ ]:
# Let's rename the columns to differentiate this feature group from existing ones in the feature store
pandas_df.columns = ["team_budget_test", "average_attendance_test"]

featurestore.create_featuregroup(
    pandas_df,
    "pandas_test_example",
    description="test featuregroup created from pandas dataframe",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)
assert "pandas_test_example_1" in featurestore.get_featuregroups()

In [ ]:
count_pre_pandas_insert_overwrite = featurestore.get_featuregroup("pandas_test_example").count()
featurestore.insert_into_featuregroup(
    pandas_df, 
    "pandas_test_example",
    descriptive_statistics=False, 
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    mode="overwrite")
count_after_pandas_insert_overwrite = featurestore.get_featuregroup("pandas_test_example").count()
assert count_pre_pandas_insert_overwrite == count_after_pandas_insert_overwrite

In [ ]:
featurestore.create_featuregroup(
    numpy_df,
    "numpy_test_example",
    description="test featuregroup created from numpy matrix",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)
assert "numpy_test_example_1" in featurestore.get_featuregroups()

In [ ]:
numpy_test_df_count_pre_insert_overwrite = featurestore.get_featuregroup("numpy_test_example", dataframe_type="spark").count()
featurestore.insert_into_featuregroup(
    numpy_df, 
    "numpy_test_example",
    descriptive_statistics=False, 
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    mode="overwrite")
numpy_test_df_count_after_insert_overwrite = featurestore.get_featuregroup("numpy_test_example", dataframe_type="spark").count()
assert numpy_test_df_count_pre_insert_overwrite == numpy_test_df_count_pre_insert_overwrite

In [ ]:
featurestore.create_featuregroup(
    python_df,
    "python_test_example",
    description="test featuregroup created from python 2D list",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)

python_test_df_count_pre_insert_overwrite = featurestore.get_featuregroup("python_test_example", dataframe_type="spark").count()
assert "python_test_example_1" in featurestore.get_featuregroups()

In [ ]:
featurestore.insert_into_featuregroup(
    python_df, 
    "python_test_example",
    descriptive_statistics=False, 
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    mode="overwrite")

python_test_df_count_after_insert_overwrite = featurestore.get_featuregroup("python_test_example", dataframe_type="spark").count()
assert python_test_df_count_pre_insert_overwrite == python_test_df_count_after_insert_overwrite

##### Test update Feature Store Statistics `featurestore.update_featuregroup_stats()`

In [ ]:
featurestore.update_featuregroup_stats("teams_features")

In [ ]:
featurestore.update_featuregroup_stats(
    "teams_features", 
    featuregroup_version=1, 
    featurestore=featurestore.project_featurestore(), 
    descriptive_statistics=True,
    feature_correlation=True, 
    feature_histograms=True,
    cluster_analysis=True,
    stat_columns=None)

##### Test Write Training Dataset Operations 

- `featurestore.get_latest_training_dataset_version()`
- `create_training_dataset()`

In [ ]:
features_df = featurestore.get_features(
    ["team_budget", "average_attendance",
    "team_position"]
)
latest_version = featurestore.get_latest_training_dataset_version("team_position_prediction")

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    training_dataset_version = 1
)

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction_csv",
    description="a dataset with features for football teams, used for training a model to predict league-position",
    featurestore=featurestore.project_featurestore(),
    data_format="csv",
    training_dataset_version= 1,
    job_name=None,
    dependencies=[],
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    stat_columns=None)

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction_tsv",
    description="a dataset with features for football teams, used for training a model to predict league-position",
    featurestore=featurestore.project_featurestore(),
    data_format="tsv",
    training_dataset_version=1,
    job_name=None,
    dependencies=[],
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    stat_columns=None)

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction_parquet",
    description="a dataset with features for football teams, used for training a model to predict league-position",
    featurestore=featurestore.project_featurestore(),
    data_format="parquet",
    training_dataset_version=1,
    job_name=None,
    dependencies=[],
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    stat_columns=None)

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction_orc",
    description="a dataset with features for football teams, used for training a model to predict league-position",
    featurestore=featurestore.project_featurestore(),
    data_format="orc",
    training_dataset_version=1,
    job_name=None,
    dependencies=[],
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    stat_columns=None)

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction_avro",
    description="a dataset with features for football teams, used for training a model to predict league-position",
    featurestore=featurestore.project_featurestore(),
    data_format="avro",
    training_dataset_version=1,
    job_name=None,
    dependencies=[],
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    stat_columns=None)

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction_hdf5",
    description="a dataset with features for football teams, used for training a model to predict league-position",
    featurestore=featurestore.project_featurestore(),
    data_format="hdf5",
    training_dataset_version=1,
    job_name=None,
    dependencies=[],
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    stat_columns=None)

In [ ]:
featurestore.create_training_dataset(
    features_df, "team_position_prediction_npy",
    description="a dataset with features for football teams, used for training a model to predict league-position",
    featurestore=featurestore.project_featurestore(),
    data_format="npy",
    training_dataset_version=1,
    job_name=None,
    dependencies=[],
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    stat_columns=None)

In [ ]:
# Petastorm is only supported in python 3
if sys.version_info[0] >= 3:
    PetastormSchema = Unischema('team_position_prediction_petastorm_schema', [
        UnischemaField('team_budget', np.float32, (), ScalarCodec(FloatType()), False),
        UnischemaField('average_attendance', np.float32, (), ScalarCodec(FloatType()), False),
        UnischemaField('team_position', np.int32, (), ScalarCodec(IntegerType()), False)
    ])

    petastorm_args = {
        "schema": PetastormSchema
    }

    featurestore.create_training_dataset(
        features_df, "team_position_prediction_petastorm",
        description="a dataset with features for football teams, used for training a model to predict league-position",
        featurestore=featurestore.project_featurestore(),
        data_format="petastorm",
        training_dataset_version=1,
        job_name=None,
        dependencies=[],
        descriptive_statistics=False,
        feature_correlation=False,
        feature_histograms=False,
        cluster_analysis=False,
        stat_columns=None,
        petastorm_args=petastorm_args
    )

In [ ]:
tds = featurestore.get_training_datasets()
assert 'team_position_prediction_1' in tds
assert 'team_position_prediction_csv_1' in tds
assert 'team_position_prediction_tsv_1' in tds
assert 'team_position_prediction_parquet_1' in tds
assert 'team_position_prediction_orc_1' in tds
assert 'team_position_prediction_avro_1' in tds
assert 'team_position_prediction_hdf5_1'in tds
assert 'team_position_prediction_npy_1' in tds
if sys.version_info[0] >= 3:
    assert 'team_position_prediction_petastorm_1' in tds

##### Test Insert into an existing training dataset, `featurestore.insert_into_training_dataset()`

In [ ]:
count_pre_insert = featurestore.get_training_dataset("team_position_prediction_csv").count()
featurestore.insert_into_training_dataset(
    features_df, 
    "team_position_prediction_csv",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    training_dataset_version=featurestore.get_latest_training_dataset_version("team_position_prediction_csv")
)
count_after_insert = featurestore.get_training_dataset("team_position_prediction_csv").count()
assert count_pre_insert == count_after_insert # td only support overwrites

##### Test Training Dataset Utility Methods

- `featurestore.get_training_dataset_path()`
- `featurestore.get_training_dataset_tf_record_schema`

In [ ]:
assert hdfs.project_path() in featurestore.get_training_dataset_path("team_position_prediction_csv")

In [ ]:
assert hdfs.project_name() + "_Training_Datasets" in featurestore.get_training_dataset_path("team_position_prediction_csv")

In [ ]:
assert "team_position_prediction_csv" in featurestore.get_training_dataset_path("team_position_prediction_csv")

In [ ]:
tf_schema = featurestore.get_training_dataset_tf_record_schema("team_position_prediction")
assert tf_schema == {'team_budget': tf.FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 
                     'average_attendance': tf.FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 
                     'team_position': tf.FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}

In [ ]:
features_df = featurestore.get_training_dataset("team_position_prediction")
tf_schema = featurestore.get_dataframe_tf_record_schema(features_df)
assert tf_schema == {'team_budget': tf.FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 
                     'average_attendance': tf.FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 
                     'team_position': tf.FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}

##### Test update Training Dataset stats

- `featurestore.update_training_dataset_stats()`

In [ ]:
featurestore.update_training_dataset_stats("team_position_prediction")

In [ ]:
featurestore.update_training_dataset_stats(
    "team_position_prediction", 
    training_dataset_version=1, 
    featurestore=featurestore.project_featurestore(), 
    descriptive_statistics=True,
    feature_correlation=True, 
    feature_histograms=True,
    cluster_analysis=True,
    stat_columns=None)

##### Test Read Training Datasets API `featurestore.get_training_dataset()`

In [ ]:
cols = ['team_budget', 'average_attendance', 'team_position']
tmp = featurestore.get_training_dataset("team_position_prediction_csv")
assert set(tmp.columns) == set(cols)
assert tmp.count() == 50

In [ ]:
tmp = featurestore.get_training_dataset("team_position_prediction_hdf5")
assert tmp.count() == 50

In [ ]:
if sys.version_info[0] >= 3:
    tmp = featurestore.get_training_dataset("team_position_prediction_petastorm")
    assert set(tmp.columns) == set(cols)
    assert tmp.count() == 50

In [ ]:
tmp = featurestore.get_training_dataset("team_position_prediction_avro")
assert set(tmp.columns) == set(cols)
assert tmp.count() == 50

In [ ]:
tmp = featurestore.get_training_dataset("team_position_prediction_orc")
assert set(tmp.columns) == set(cols)
assert tmp.count() == 50

In [ ]:
tmp = featurestore.get_training_dataset("team_position_prediction_tsv")
assert set(tmp.columns) == set(cols)
assert tmp.count() == 50

In [ ]:
tmp = featurestore.get_training_dataset("team_position_prediction_npy")
assert tmp.count() == 50

In [ ]:
tmp = featurestore.get_training_dataset("team_position_prediction_parquet")
assert set(tmp.columns) == set(cols)
assert tmp.count() == 50

##### Cleanup (Delete FS Contents so that next test run works the same)

In [ ]:
# Delete feature groups
spark.sql('use ' + featurestore.project_featurestore())
for fg in featurestore.get_featuregroups():
    spark.sql("drop table " + fg)

In [ ]:
# Delete training datasets
td_dir = hdfs.project_name() + "_Training_Datasets/"
for td in featurestore.get_training_datasets():
    hdfs.rmr(td_dir + td)

In [ ]:
featurestore.get_featurestore_metadata(update_cache=True)
assert featurestore.get_featuregroups() == []
assert featurestore.get_training_datasets() == []

## Kafka Tests

##### Test default config 

- `kafka.get_default_config()`, 
- `kafka.get_security_protocol()`,
- `kafka.get_broker_endpoints_list()`

In [ ]:
config = kafka.get_kafka_default_config()
assert "bootstrap.servers" in config
assert "security.protocol" in config
assert "ssl.ca.location" in config
assert "ssl.key.location" in config
assert "ssl.certificate.location" in config

In [ ]:
assert len(kafka.get_security_protocol()) > 0
assert len(kafka.get_broker_endpoints_list()) > 0

## TLS Tests

##### Test access to TLS tokens

- `tls.get_key_store()`
- `tls.get_trust_store()`
- `tls.get_key_store_pwd()`
- `tls.get_trust_store_pwd()`
- `tls.get_client_certificate_location()`
- `tls.get_client_key_location()`
- `tls.get_ca_chain_location()`

In [ ]:
assert len(tls.get_key_store()) > 0
assert len(tls.get_trust_store()) > 0
assert len(tls.get_key_store_pwd()) > 0
assert len(tls.get_trust_store_pwd()) > 0
assert len(tls.get_client_certificate_location()) > 0
assert len(tls.get_client_key_location()) > 0
assert len(tls.get_ca_chain_location()) > 0

## Serving Tests

These tests require that you have the following files in the Resources directory:

- `iris_model.knn`
- `iris_flower_classifier.py`
- `mnist`

Where mnist is a directory containing a tensorflow model.

These files can be downloaded from here: `http://snurran.sics.se/hops/hops-util-py_test/`

##### Test Export Model

In [2]:
model_path = "Resources/iris_knn.pkl"
serving.export(model_path, "IrisFlowerClassifier", 1, overwrite=True)
assert hdfs.exists("Models/IrisFlowerClassifier/1/iris_knn.pkl")

In [3]:
model_path = "Resources/iris_flower_classifier.py"
serving.export(model_path, "IrisFlowerClassifier", 1, overwrite=True)
assert hdfs.exists("Models/IrisFlowerClassifier/1/iris_flower_classifier.py")

In [4]:
model_path = "Resources/mnist/"
serving.export(model_path, "mnist", 2, overwrite=True)
assert hdfs.exists("Models/mnist/2/")

##### Test Serve Model

In [5]:
script_path = "Models/IrisFlowerClassifier/1/iris_flower_classifier.py"
if serving.exists("IrisFlowerClassifier"):
    serving.delete("IrisFlowerClassifier")
serving.create_or_update(script_path, "IrisFlowerClassifier", serving_type="SKLEARN", 
                                 model_version=1)

Deleting serving with name: IrisFlowerClassifier...
Serving with name: IrisFlowerClassifier successfully deleted
Creating a serving for model IrisFlowerClassifier ...
Serving for model IrisFlowerClassifier successfully created

In [6]:
assert serving.exists("IrisFlowerClassifier")

In [7]:
model_path = "Models/mnist/2/"
if serving.exists("mnist"):
    serving.delete("mnist")
serving.create_or_update(model_path, "mnist", serving_type="TENSORFLOW", 
                                 model_version=2)

No serving with name mnist was found in the project demo_deep_learning_admin000
Creating a serving for model mnist ...
Serving for model mnist successfully created

In [8]:
assert serving.exists("mnist")

##### Test Data Access Operations on Model

In [10]:
assert serving.get_id("IrisFlowerClassifier") is not None
assert serving.get_id("mnist") is not None
assert "Models/IrisFlowerClassifier/1/iris_flower_classifier.py" in serving.get_artifact_path("IrisFlowerClassifier")
assert "Models/mnist/2/" in serving.get_artifact_path("mnist")
assert serving.get_type("IrisFlowerClassifier") == "SKLEARN"
assert serving.get_type("mnist") == "TENSORFLOW"
assert serving.get_version("IrisFlowerClassifier") == 1
assert serving.get_version("mnist") == 2
assert serving.get_kafka_topic("IrisFlowerClassifier") is not None
assert serving.get_kafka_topic("mnist") is not None
assert serving.get_status("IrisFlowerClassifier") == "Stopped"
assert serving.get_status("mnist") == "Stopped"

##### Test Start/Stop Serving

In [11]:
serving.start("IrisFlowerClassifier")
serving.start("mnist")

Starting serving with name: IrisFlowerClassifier...
Serving with name: IrisFlowerClassifier successfully started
Starting serving with name: mnist...
Serving with name: mnist successfully started

In [12]:
assert serving.get_status("IrisFlowerClassifier") == "Running"
assert serving.get_status("mnist") == "Running"

In [13]:
serving.stop("IrisFlowerClassifier")
serving.stop("mnist")

Stopping serving with name: IrisFlowerClassifier...
Serving with name: IrisFlowerClassifier successfully stopped
Stopping serving with name: mnist...
Serving with name: mnist successfully stopped

In [14]:
assert serving.get_status("IrisFlowerClassifier") == "Stopped"
assert serving.get_status("mnist") == "Stopped"

##### Test Send Inference Requests

In [15]:
serving.start("IrisFlowerClassifier")
serving.start("mnist")

Starting serving with name: IrisFlowerClassifier...
Serving with name: IrisFlowerClassifier successfully started
Starting serving with name: mnist...
Serving with name: mnist successfully started

In [16]:
for i in range(20):
    data = {"inputs" : [[random.uniform(1, 8) for i in range(4)]]}
    response = serving.make_inference_request("IrisFlowerClassifier", data)
    assert response is not None
    assert "predictions" or "prediction" in response

In [17]:
for i in range(20):
    data = {
                "signature_name": 'predict_images',
                "instances": [np.random.rand(784).tolist()]
            }
    response = serving.make_inference_request("mnist", data)
    assert response is not None
    assert "predictions" in response

##### Test Kafka Inference Log

In [18]:
topic = serving.get_kafka_topic("IrisFlowerClassifier")
config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config)
topics = [topic]
consumer.subscribe(topics)
json_schema = kafka.get_schema(topic)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

In [19]:
for i in range(0, 10):
    msg = consumer.poll(timeout=1.5)
    if msg is not None:
        value = msg.value()
        event_dict = kafka.parse_avro_msg(value, avro_schema)
        assert "modelName" in event_dict
        assert "requestTimestamp" in event_dict
        assert "servingType" in event_dict
        assert "inferenceResponse" in event_dict
        assert event_dict["modelName"] == "IrisFlowerClassifier"
        assert event_dict["servingType"] == "SKLEARN"

In [20]:
topic = serving.get_kafka_topic("mnist")
config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config)
topics = [topic]
consumer.subscribe(topics)
json_schema = kafka.get_schema(topic)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

In [21]:
for i in range(0, 10):
    msg = consumer.poll(timeout=1.5)
    if msg is not None:
        value = msg.value()
        event_dict = kafka.parse_avro_msg(value, avro_schema)
        assert "modelName" in event_dict
        assert "requestTimestamp" in event_dict
        assert "servingType" in event_dict
        assert "inferenceResponse" in event_dict
        assert event_dict["modelName"] == "mnist"
        assert event_dict["servingType"] == "TENSORFLOW"

##### Test Delete Serving

In [22]:
serving.delete("IrisFlowerClassifier")
serving.delete("mnist")

Deleting serving with name: IrisFlowerClassifier...
Serving with name: IrisFlowerClassifier successfully deleted
Deleting serving with name: mnist...
Serving with name: mnist successfully deleted

In [23]:
assert not serving.exists("IrisFlowerClassifier")
assert not serving.exists("mnist")

No serving with name IrisFlowerClassifier was found in the project demo_deep_learning_admin000
No serving with name mnist was found in the project demo_deep_learning_admin000